In [4]:
#@title Requirements

import tensorflow as tf


In [5]:
#@title Choosing Resources

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
   tf.config.experimental.set_memory_growth(gpu, True)

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1./256,
    brightness_range=(0.7,1.2),
    horizontal_flip=True
)

# Load dữ liệu từ thư mục
train_generator = datagen.flow_from_directory(
    'Dataset_img/train',
    target_size=(256, 256),
    batch_size=350,
    class_mode='binary')

test_generator = datagen.flow_from_directory(
    'Dataset_img/test',
    target_size=(256, 256),
    batch_size=180,
    class_mode='binary')


val_generator = datagen.flow_from_directory(
    'Dataset_img/val',
    target_size=(256, 256),
    batch_size=90,
    class_mode='binary')



Found 775 images belonging to 3 classes.
Found 216 images belonging to 3 classes.
Found 115 images belonging to 3 classes.


In [2]:
#@title Deep Models XceptionNet, InceptionResnetV2, MobileNetV2

''' Deep Model '''
# The Network
import keras.backend as K
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Dense, Flatten, Dropout, AveragePooling2D, Concatenate, GlobalAveragePooling2D, BatchNormalization, ReLU, Add, SeparableConv2D
from tensorflow.keras.applications import MobileNetV2, InceptionResNetV2, Xception


def xception(img_shape, n_classes):
    xceptionnet = Xception(input_shape=img_shape, include_top=False, weights='imagenet')
    xceptionnet.trainable = False

    input = Input(img_shape)
    x = xceptionnet(input, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(rate = 0.2)(x)

    output = Dense(n_classes, activation='softmax')(x)

    model = Model(input, output)
    return model





def inceptionresnetv2(img_shape, n_classes):
    inceptionresnet = InceptionResNetV2(input_shape=img_shape, include_top=False, weights='imagenet')
    inceptionresnet.trainable = False

    input = Input(img_shape)
    x = inceptionresnet(input, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(rate = 0.2)(x)

    output = Dense(n_classes, activation='softmax')(x)

    model = Model(input, output)
    return model







def mobilenetv2(img_shape, n_classes):
    mobilenet = MobileNetV2(input_shape=img_shape, include_top=False, weights='imagenet') # include_top: Một cờ chỉ định xem liệu bạn muốn bao gồm các lớp fully connected (hay "top") cho việc phân loại cuối cùng hay không. Trong trường hợp này, include_top=False cho biết rằng chúng ta không bao gồm các lớp fully connected.
    # 
    # weights: Có thể là None (không sử dụng trọng số được đào tạo trước), 'imagenet' (sử dụng trọng số được đào tạo trước trên tập dữ liệu ImageNet), hoặc đường dẫn đến file chứa trọng số.

    mobilenet.trainable = False
    # Việc đặt trainable thành False là phổ biến khi sử dụng các mô hình được đào tạo trước như MobileNetV2 như một bộ trích xuất đặc trưng trong một mô hình lớn hơn. Bằng cách này, chúng ta chỉ muốn sử dụng các trọng số được đào tạo trước mà không muốn cập nhật chúng trong quá trình huấn luyện mô hình toàn bộ. Điều này giúp tiết kiệm thời gian huấn luyện và giảm nguy cơ overfitting.
    input = Input(img_shape)
    x = mobilenet(input, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(rate = 0.2)(x)

    output = Dense(n_classes, activation='softmax')(x)

    model = Model(input, output)
    return model




input_shape = (256, 256, 3)
n_classes = 3

from keras.optimizers import SGD
opt = SGD(learning_rate=0.1)

model1 = xception(input_shape, n_classes)
model1.compile('Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model1.summary()

model2 = inceptionresnetv2(input_shape, n_classes)
model2.compile('Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model2.summary()

model3 = mobilenetv2(input_shape, n_classes)
model3.compile('Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model3.summary()

# Trong hàm model1.compile() trong mã trên, Adam được sử dụng làm tối ưu hóa. Adam là một phương pháp tối ưu hóa gradient descent được sử dụng phổ biến trong học sâu. Đây là một thuật toán kết hợp giữa hai phương pháp tối ưu hóa: AdaGrad và RMSProp.

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ xception (Functional)           │ (None, 8, 8, 2048)     │    20,861,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3)              │         6,147 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,867,627 (79.60 MB)

 Trainable params: 6,147 (24.01 KB)

 Non-trainable params: 20,861,480 (79.58 MB)

...

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 8, 8, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │         3,843 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,261,827 (8.63 MB)

 Trainable params: 3,843 (15.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [6]:
#@title Training the Models

# Train
logdir='logs'
tensorboard_callback1 = tf.keras.callbacks.TensorBoard(log_dir=logdir)
hist1 = model1.fit(train_generator,epochs=60, validation_data=val_generator, callbacks=[tensorboard_callback1])
print(hist1.history)

tensorboard_callback2 = tf.keras.callbacks.TensorBoard(log_dir=logdir)
hist2 = model2.fit(train_generator, epochs=60, validation_data=val_generator, callbacks=[tensorboard_callback2])
print(hist2.history)

tensorboard_callback3 = tf.keras.callbacks.TensorBoard(log_dir=logdir)
hist3 = model3.fit(train_generator, epochs=60, validation_data=val_generator, callbacks=[tensorboard_callback3])
print(hist3.history)

Epoch 1/60


c:\Users\ADmin\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3/3 ━━━━━━━━━━━━━━━━━━━━ 159s 31s/step - accuracy: 0.4044 - loss: 1.0478 - val_accuracy: 0.4957 - val_loss: 1.0507
Epoch 2/60
3/3 ━━━━━━━━━━━━━━━━━━━━ 119s 27s/step - accuracy: 0.6468 - loss: 0.8354 - val_accuracy: 0.5652 - val_loss: 0.9428
Epoch 3/60
3/3 ━━━━━━━━━━━━━━━━━━━━ 123s 31s/step - accuracy: 0.7697 - loss: 0.6950 - val_accuracy: 0.5652 - val_loss: 0.8364
Epoch 4/60
3/3 ━━━━━━━━━━━━━━━━━━━━ 156s 31s/step - accuracy: 0.7867 - loss: 0.6286 - val_accuracy: 0.6435 - val_loss: 0.8461
Epoch 5/60
3/3 ━━━━━━━━━━━━━━━━━━━━ 125s 25s/step - accuracy: 0.8167 - loss: 0.5391 - val_accuracy: 0.6522 - val_loss: 0.7835
Epoch 6/60
3/3 ━━━━━━━━━━━━━━━━━━━━ 170s 44s/step - accuracy: 0.8168 - loss: 0.4795 - val_accuracy: 0.6174 - val_loss: 0.8237
Epoch 7/60
3/3 ━━━━━━━━━━━━━━━━━━━━ 147s 67s/step - accuracy: 0.8386 - loss: 0.4446 - val_accuracy: 0.6783 - val_loss: 0.8472
Epoch 8/60
3/3 ━━━━━━━━━━━━━━━━━━━━ 112s 26s/step - accuracy: 0.8528 - loss: 0.3763 - val_accuracy: 0.7043 - val_loss: 0.7702
Epo